In [1]:
from moviepy.editor import AudioFileClip, concatenate_audioclips
from scipy.io import wavfile
import numpy as np
import itertools
import os

print('OK !')


Bad key "text.kerning_factor" on line 4 in
/Users/thanatchonchongmankhong/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
OK !


In [2]:
def improveloop(x0, a, b, sr=44100, w1=0.010, w2=0.100):
    """
    Input:  (a, b) is a loop
    Output: (a, B) is a better loop 
            distance (the less the distance the better the loop)
    This function moves the loop's endpoint b to B (up to 100 ms further) such that (a, B) is a "better" loop, i.e. sum((x0[a:a+10ms] - x0[B:B+10ms])^2) is minimal
    """

    W1 = int(w1*sr)
    W2 = int(w2*sr)
    x = x0[a:a+W1]
    y = x0[b:b+W2]
    delta = np.sum(x**2) - 2*np.correlate(y, x) + np.correlate(y**2, np.ones_like(x))
    K = np.argmin(delta)
    B = K + b
    distance = delta[K]

    return a, B, distance

def seamless_loop(wave_file, shortest_th = 0):
    sr, x = wavfile.read(wave_file)
    x0 = x if x.ndim == 1 else x[:, 0]
    x0 = np.asarray(x0, dtype=np.float32)
    
    error = False
    times = 0
    while not error:
        times+=1
        try:
            A = [int((2 + 0.25 * k) * sr) for k in range(times)]
            dist = np.inf
            for a, b in itertools.product(A, A):  # cartesian product: pairs (a, b) of points on the grid
                if b - a < 1 * sr:
                    continue
                a, B, d = improveloop(x0, a, b, sr=sr)
                if d < dist:
                    aa = a
                    BB = B
                    dist = d
        except:
            error = True

    valid_times = times-1
    d_dict = {}
    d_list = []

    A = [int((2 + 0.25 * k) * sr) for k in range(valid_times)]
    dist = np.inf
    for a, b in itertools.product(A, A):  # cartesian product: pairs (a, b) of points on the grid
        if b - a < 1 * sr:
            continue
        a, B, d = improveloop(x0, a, b, sr=sr)
#         print( 'Loop (%.3f, %.3f) improved to (%.3fs, %.3fs), distance: %i' % 
#             (a * 1.0 / sr, b * 1.0 / sr, a * 1.0 / sr, B * 1.0 / sr, d))
        d_list.append(d)
        d_dict[d] = (a * 1.0 / sr,B * 1.0 / sr)

        if d < dist:
            aa = a
            BB = B
            dist = d
    d_list.sort()
    return d_dict[d_list[shortest_th]]
print('OK !')

OK !


In [3]:
def wave_extraction(start_sec, end_sec, export_file, seconds):
    audio_file = "data/Neorhythm - User Guide - How to use Test Tube.webm"
    audioclip = AudioFileClip(audio_file)

    enhance_mental_capacity = audioclip.subclip(start_sec, end_sec)
    # enhance_mental_capacity.preview()
    enhance_mental_capacity.write_audiofile("data/temp.wav")
    loop_interval = seamless_loop("data/temp.wav")
    os.remove("data/temp.wav")

    seemless_enhance_mental_capacity = enhance_mental_capacity.subclip(loop_interval[0], loop_interval[1])
#     seemless_enhance_mental_capacity.preview()
    loop_no = int((seconds / seemless_enhance_mental_capacity.duration) + 1)
    loop_clip = [seemless_enhance_mental_capacity for each in range(loop_no)]
    full_clip = concatenate_audioclips(loop_clip)

    full_clip.write_audiofile(export_file)
    audioclip.close()
    print('Export Completed !')
print('OK !')

OK !


In [4]:
#enhance_mental_capacity
%time wave_extraction(56.0, 62.5,"data/enhance_mental_capacity.mp3", 30)

MoviePy - Writing audio in data/temp.wav


MoviePy - Done.


chunk:   6%|▌         | 38/681 [00:00<00:02, 320.92it/s, now=None]

MoviePy - Writing audio in data/enhance_mental_capacity.mp3


MoviePy - Done.
Export Completed !
CPU times: user 1.3 s, sys: 122 ms, total: 1.42 s
Wall time: 3.18 s


In [5]:
#energy_and_vitality
%time wave_extraction(66.4, 71.8,"data/energy_and_vitality.mp3", 30)

chunk:   0%|          | 0/120 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/temp.wav


MoviePy - Done.


chunk:   6%|▌         | 38/685 [00:00<00:02, 310.26it/s, now=None]

MoviePy - Writing audio in data/energy_and_vitality.mp3


MoviePy - Done.
Export Completed !
CPU times: user 1.06 s, sys: 119 ms, total: 1.18 s
Wall time: 3.07 s


In [6]:
#deep_relaxation
%time wave_extraction(76.5, 82.5,"data/deep_relaxation.mp3", 30)

chunk:   0%|          | 0/133 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/temp.wav


MoviePy - Done.


chunk:   6%|▌         | 38/665 [00:00<00:02, 282.76it/s, now=None]

MoviePy - Writing audio in data/deep_relaxation.mp3


MoviePy - Done.
Export Completed !
CPU times: user 1.25 s, sys: 127 ms, total: 1.37 s
Wall time: 3.51 s


In [7]:
#improved_sleep
%time wave_extraction(87, 92,"data/improved_sleep.mp3", 30)

MoviePy - Writing audio in data/temp.wav
MoviePy - Done.


chunk:   6%|▌         | 38/668 [00:00<00:02, 260.86it/s, now=None]

MoviePy - Writing audio in data/improved_sleep.mp3


MoviePy - Done.
Export Completed !
CPU times: user 1.02 s, sys: 239 ms, total: 1.26 s
Wall time: 3.77 s


In [8]:
#pain_control
%time wave_extraction(97.5, 100.7,"data/pain_control.mp3", 30)

chunk:   0%|          | 0/71 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/temp.wav


MoviePy - Done.
MoviePy - Writing audio in data/pain_control.mp3


MoviePy - Done.
Export Completed !
CPU times: user 1.07 s, sys: 139 ms, total: 1.21 s
Wall time: 3.21 s


In [9]:
#meditation_theta
%time wave_extraction(106, 111,"data/meditation_theta.mp3", 30)

chunk:   0%|          | 0/111 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/temp.wav


chunk:   0%|          | 0/672 [00:00<?, ?it/s, now=None]         

MoviePy - Done.
MoviePy - Writing audio in data/meditation_theta.mp3


MoviePy - Done.
Export Completed !
CPU times: user 954 ms, sys: 107 ms, total: 1.06 s
Wall time: 2.58 s


In [10]:
#meditation_calming_synchronization
%time wave_extraction(117, 121,"data/meditation_calming_synchronization.mp3", 30)

chunk:   0%|          | 0/677 [00:00<?, ?it/s, now=None]        

MoviePy - Writing audio in data/temp.wav
MoviePy - Done.
MoviePy - Writing audio in data/meditation_calming_synchronization.mp3


MoviePy - Done.
Export Completed !
CPU times: user 950 ms, sys: 122 ms, total: 1.07 s
Wall time: 2.63 s
